<a href="https://colab.research.google.com/github/SereniT33/Omdena_Disaster_response_ai/blob/main/Useful_codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Useful code snippets

### Load the Dataset

In [ ]:
DATADIR= "...file path..."
CATEGORIES=["Damaged","Undamaged"]
#Iterate through the two categories
for category in CATEGORIES:
    path= os.path.join(DATADIR,category)
    for img in os.listdir(path):
        img_array= cv2.imread(os.path.join(path,img))
        plt.imshow(img_array)
        plt.show()
        break
    break

### Read or resize the image

In [ ]:
# code snippet 1
import cv2

# Load the image
image = cv2.imread('example.jpg')

# Resize the image to a specific width and height
width = 300
height = 200
resized_image = cv2.resize(image, (width, height))

In [ ]:
#Resizing all images to one size
training_data=[]
def create_training_data():
    for category in CATEGORIES:
        path= os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array= cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()
#Checking the new image size
new_array.shape

### Train the model (CNN)

In [ ]:
#Creating features and labels for the images in our dataset
X_calamity=[]#features
y_calamity=[]#labels
for features, label in training_data:
    X_calamity.append(features)
    y_calamity.append(label)
#Converting image input to array
X_calamity= np.array(X_calamity).reshape(-3,IMG_SIZE,IMG_SIZE,3)
#Normalising the image input array
X_calamity=X_calamity/255.0

### Formulate the model

In [ ]:
model = Sequential()
#Second Convolutional layer
model.add(Conv2D(kernel_size=3, strides=1, filters=64, padding='same', activation='relu', name='layer_conv2'))
model.add(MaxPooling2D(pool_size=(2,2)))
#Third Convolutional layer
model.add(Conv2D(kernel_size=3, strides=1, filters=128, padding='same', activation='relu', name='layer_conv3'))
model.add(MaxPooling2D(pool_size=(2,2)))
#Drop-out layer
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=2, strides=2))
#Fourth Convolutional layer
model.add(Conv2D(kernel_size=3, strides=1, filters=64, padding='same', activation='relu', name='layer_conv4'))
model.add(MaxPooling2D(pool_size=(2,2)))
#Fifth Convolutional layer
model.add(Conv2D(kernel_size=3, strides=1, filters=32, padding='same', activation='relu', name='layer_conv5'))
model.add(MaxPooling2D(pool_size=(2,2)))
#Sixth Convolutional layer
model.add(Conv2D(kernel_size=3, strides=1, filters=16, padding='same', activation='relu', name='layer_conv6'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(32)) #Fully Connected Layer
model.add(Activation("relu"))
#Output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss="binary_crossentropy",
             optimizer="adam", metrics=['accuracy'])
#Displaying model summary statistics
model.summary()

### Results

In [ ]:
model.fit(X_calamity, y_calamity, batch_size=65, epochs=20, validation_split=0.1)

### Visualize the results

In [ ]:
#Helper Function to plot images
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9

    # Create figure with 3x3 sub-plots
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)
for i, ax in enumerate(axes.flat):
        # Plot image
        ax.imshow(images[i].reshape(image_shape), cmap='binary')
#Show true and predicted classes
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])
#Show the classes as the label on the x-axis
        ax.set_xlabel(xlabel)

        # Remove ticks from the plot
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

##### Plot images and labels

In [ ]:
#Defining parameters
images = X_calamity[0:9]
cls_true =y_calamity[0:9]
y_pred = model.predict(x=images)
cls_pred = np.argmax(y_pred, axis=1)
#Plotting images
plot_images(images=images, cls_true=cls_true, cls_pred=cls_pred)

In [ ]:
#Helper Function to plot incorrect images
def plot_example_errors(cls_pred):
#cls_pred is an array of the predicted class-number for all images in the test-set
# Boolean array whether the predicted class is incorrect.
    incorrect = (cls_pred != y_calamity)
# Get the images from the test-set that have been incorrectly classified
    images = X_calamity[incorrect]

    #Get the predicted classes for those images
    cls_pred = cls_pred[incorrect]
#Get the true classes for those images
    cls_true = y_calamity[incorrect]

    #Plot the first 9 images
    plot_images(images=images[0:9], cls_true=cls_true[0:9], cls_pred=cls_pred[0:9])

In [ ]:
#Defining parameters
y_pred = model.predict(x=X_calamity)
cls_pred = np.argmax(y_pred, axis=1)
#Plotting images
plot_example_errors(cls_pred)